2 stage modelling:redifining imu

In [7]:
def imu_data(emotion):
    if emotion == 0:   # Neutral
        ax, ay, az = np.random.uniform(-0.5, 0.5, 3)
        gx, gy, gz = np.random.uniform(-5, 5, 3)

    elif emotion == 1: # Happy (light motion)
        ax, ay, az = np.random.uniform(-1, 1, 3)
        gx, gy, gz = np.random.uniform(-20, 20, 3)

    else:              # Angry (aggressive motion)
        ax, ay, az = np.random.uniform(-3, 3, 3)
        gx, gy, gz = np.random.uniform(-80, 80, 3)
    return [ax, ay, az, gx, gy, gz]

# Also redefine gesture_map to ensure it matches the trained model (4 classes)
gesture_map = {
    0: "Open Hand",
    1: "Fist",
    2: "Pointing",
    3: "Pinch" # Added for 4th class
}

# Example real-time input
flex_angles_input = [10, 15, 80, 85, 80] # Example for 'Pointing' gesture
pressure_input = [70, 65, 30, 90] # Example for 'Press Object' scenario
emotion_type_for_imu = 2 # Example: Angry motion

# Convert raw sensor data to voltages/readings
current_flex_vals = [flex_voltage(a) for a in flex_angles_input]
current_press_vals = [pressure_voltage(p) for p in pressure_input]
current_imu_vals = imu_data(emotion_type_for_imu) # returns [ax, ay, az, gx, gy, gz]

# Calculate engineered features for this single instance
# For a single timestep, Gesture_Speed is interpreted as variance across flex sensors.
gesture_speed_feature = np.var(current_flex_vals)
pressure_variance_feature = np.var(current_press_vals)
motion_magnitude_feature = np.sqrt(current_imu_vals[0]**2 + current_imu_vals[1]**2 + current_imu_vals[2]**2)

# Prepare input for gesture model (LSTM)
# The LSTM was trained on (samples, 1, features)
sensor_input = np.array([[[gesture_speed_feature, pressure_variance_feature, motion_magnitude_feature]]])
gesture_probs = model_gesture.predict(sensor_input)
gesture_id = np.argmax(gesture_probs)

gesture_text = gesture_map[gesture_id]

# Gesture intensity (confidence)
gesture_intensity = np.max(gesture_probs)

# Stage 2 Prediction
# Prepare input for emotion model (SVM)
# SVM was trained on (samples, features) and scaled.
# The original Xe_train features were 'Gesture_Speed', 'Pressure_Variance', 'Motion_Magnitude'
emotion_input_features = np.array([[gesture_speed_feature, pressure_variance_feature, motion_magnitude_feature]])
emotion_input_scaled = scaler.transform(emotion_input_features) # Use the scaler saved from the emotion model training
emotion_id = emotion_model.predict(emotion_input_scaled)[0]
emotion = emotion_map[emotion_id]

print("Predicted Gesture:", gesture_text, "| Emotion:", emotion)

Predicted Gesture: Pointing | Emotion: Angry
